<a href="https://colab.research.google.com/github/wenjunsun/Covid-19-analysis-with-uw-ubicomp/blob/master/2020-10/standardized_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will use not just a normalized linear regression but standardized linear regression, which is one of the possible methods to detect the importance of features in impacting the respnose variable. (response in this case is the change in people's stay at home behavior during COVID)

*Data used here use a time window from shelter in place date to 6/1, and use shetler in place date as the baseline date, not first case date. The counties without shelter in place uses mean shelter in place date.*

In [1]:
cd drive/My\ Drive/fall

/content/drive/My Drive/fall


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('county_data_with_reduced_covariates_with_SIP.csv')

In [4]:
data.head()

,state,state_code,State Name,cnamelong,county_code,diff_in_perc_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,AL,Alabama,Autauga County,1001.0,0.050678,1,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,AL,Alabama,Baldwin County,1003.0,0.050312,1,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,AL,Alabama,Barbour County,1005.0,0.007037,1,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,AL,Alabama,Bibb County,1007.0,0.011809,1,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,AL,Alabama,Blount County,1009.0,0.038890,1,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9


In [6]:
# drop non-numeric columns
data.drop(['state_code', 'State Name', 'cnamelong'], axis = 1, inplace=True)

In [15]:
data.shape

(2996, 14)

In [16]:
data.dropna(inplace=True)

In [17]:
data.shape

(2995, 14)

We first do a linear regression of all the counties, then do a separate regression analysis on the counties with/without shelter in place.

# 1. Standardized Linear Regression on all counties

In [18]:
# get covariates
X = data.drop(['diff_in_perc_at_home', 'SIP?'], axis=1)
# get response variable
y = data['diff_in_perc_at_home'] 

In [19]:
X

,state,county_code,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,1001.0,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,1003.0,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,1005.0,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,1007.0,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,1009.0,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9
...,...,...,...,...,...,...,...,...,...,...,...,...
2991,56.0,56037.0,73315.0,10.9,43051,-0.535382,26.2,12.1,1.0,1.1,16.1,79.3
2992,56.0,56039.0,99087.0,46.4,23081,0.278663,18.4,15.4,1.4,0.6,14.9,81.5
2993,56.0,56041.0,63401.0,43.1,20299,-0.614926,28.8,14.1,0.5,0.7,9.2,87.4
2994,56.0,56043.0,55190.0,36.0,7885,-0.640377,22.7,21.7,0.8,0.5,14.1,82.4


In [20]:
y

0       0.050678
1       0.050312
2       0.007037
3       0.011809
4       0.038890
          ...   
2991   -0.018485
2992    0.075183
2993    0.010157
2994   -0.007825
2995   -0.003361
Name: diff_in_perc_at_home, Length: 2995, dtype: float64

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [23]:
regressor = LinearRegression().fit(X_scaled, y)
r_2 = regressor.score(X_scaled, y) # R^2
print(f'R^2 score is {r_2}')

R^2 score is 0.6204935097006014


In [24]:
regressor.coef_

array([-0.11324832,  0.11216252,  0.0268063 , -0.00489883,  0.00593943,
        0.00166582, -0.00249897,  0.00134661,  0.00254383,  0.0070198 ,
        0.00277878,  0.00040917])

In [26]:
X.columns[0]

'state'

In [28]:
for i, coef in enumerate(regressor.coef_):
  print(f'after standardization, feature {X.columns[i]}\' coefficient is {coef}')

after standardization, feature state' coefficient is -0.11324831882999457
after standardization, feature county_code' coefficient is 0.1121625203982438
after standardization, feature Median Household Income' coefficient is 0.026806300144680044
after standardization, feature % Rural' coefficient is -0.004898829211590388
after standardization, feature Population_y' coefficient is 0.005939431320570068
after standardization, feature political_diff' coefficient is 0.0016658248904302276
after standardization, feature % less than 18 years of age' coefficient is -0.0024989651619581883
after standardization, feature % 65 and over' coefficient is 0.0013466138402318173
after standardization, feature % Asian' coefficient is 0.0025438326840402457
after standardization, feature % Black' coefficient is 0.007019799750249989
after standardization, feature % Hispanic' coefficient is 0.002778779318194798
after standardization, feature % Non-Hispanic White' coefficient is 0.00040917003479432317


As we can see, the result is not so trustworthy because state + county_code has a big coefficient (I included them to see what random feature's coefficients will be). But we can still see within the valid covariates that median household income has a lot of weight.

Now let's do increase of R^2 analysis, when we include more and more covariates.

In [44]:
X_scaled[:,:1]

array([[-1.93227292],
       [-1.93227292],
       [-1.93227292],
       ...,
       [ 1.70092599],
       [ 1.70092599],
       [ 1.70092599]])

In [46]:
r_2 = 0
for i, column in enumerate(X.columns):
  regressor = LinearRegression().fit(X_scaled[:,:i+1], y) # get the first i columns of X
  r_2_new = regressor.score(X_scaled[:,:i+1], y) # the new R^2 we get by including one more covariate
  print(f'including column {column} increase R^2 by {r_2_new - r_2}')
  r_2 = r_2_new

including column state increase R^2 by 0.0004957572833375057
including column county_code increase R^2 by 4.516964535627999e-06
including column Median Household Income increase R^2 by 0.5070768643664485
including column % Rural increase R^2 by 0.046398691366300926
including column Population_y increase R^2 by 0.030175535546876242
including column political_diff increase R^2 by 0.018466379457459325
including column % less than 18 years of age increase R^2 by 0.002147545561360209
including column % 65 and over increase R^2 by 1.6201555181893923e-06
including column % Asian increase R^2 by 0.0013406707188996503
including column % Black increase R^2 by 0.011879686488134977
including column % Hispanic increase R^2 by 0.0024981447140786095
including column % Non-Hispanic White increase R^2 by 8.097077651592066e-06


We can see the increase of R^2 that is the biggest is median household income? **It increases the explanatory power of the model by 50% when we include it**????!!!! 

# 2. Standardized Linear Regression on counties with shelter in place

In [48]:
data_SIP = data[data['SIP?'] == 1]

In [49]:
data_SIP

,state,county_code,diff_in_perc_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,1001.0,0.050678,1,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,1003.0,0.050312,1,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,1005.0,0.007037,1,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,1007.0,0.011809,1,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,1009.0,0.038890,1,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,55.0,55135.0,0.018980,1,58648.0,64.9,51128,-0.302460,20.3,20.9,0.6,0.5,3.3,94.1
2971,55.0,55137.0,-0.019303,1,52340.0,89.5,24263,-0.323301,18.3,24.4,0.6,2.0,6.3,89.6
2972,55.0,55139.0,0.071851,1,57785.0,13.4,171020,-0.074431,20.5,16.4,3.0,2.2,4.2,88.4
2973,55.0,55141.0,0.007077,1,55879.0,36.7,73055,-0.192777,21.6,20.6,2.0,0.9,3.1,92.2


In [50]:
# get covariates
X = data_SIP.drop(['diff_in_perc_at_home', 'SIP?'], axis=1)
# get response variable
y = data_SIP['diff_in_perc_at_home'] 

In [51]:
X

,state,county_code,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
0,1.0,1001.0,59338.0,42.0,55601,-0.494789,23.7,15.6,1.2,19.3,3.0,74.3
1,1.0,1003.0,57588.0,42.3,218022,-0.577862,21.6,20.4,1.2,8.8,4.6,83.1
2,1.0,1005.0,34382.0,67.8,24881,-0.056112,20.9,19.4,0.5,48.0,4.3,45.6
3,1.0,1007.0,46064.0,68.4,22400,-0.555441,20.5,16.5,0.2,21.1,2.6,74.6
4,1.0,1009.0,50412.0,90.0,57840,-0.813820,23.2,18.2,0.3,1.5,9.6,86.9
...,...,...,...,...,...,...,...,...,...,...,...,...
2970,55.0,55135.0,58648.0,64.9,51128,-0.302460,20.3,20.9,0.6,0.5,3.3,94.1
2971,55.0,55137.0,52340.0,89.5,24263,-0.323301,18.3,24.4,0.6,2.0,6.3,89.6
2972,55.0,55139.0,57785.0,13.4,171020,-0.074431,20.5,16.4,3.0,2.2,4.2,88.4
2973,55.0,55141.0,55879.0,36.7,73055,-0.192777,21.6,20.6,2.0,0.9,3.1,92.2


In [52]:
y

0       0.050678
1       0.050312
2       0.007037
3       0.011809
4       0.038890
          ...   
2970    0.018980
2971   -0.019303
2972    0.071851
2973    0.007077
2992    0.075183
Name: diff_in_perc_at_home, Length: 2548, dtype: float64

In [53]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [54]:
regressor = LinearRegression().fit(X_scaled, y)
r_2 = regressor.score(X_scaled, y) # R^2
print(f'R^2 score is {r_2}')

R^2 score is 0.6619202951958637


In [55]:
regressor.coef_

array([ 0.00724313, -0.00768043,  0.02848316, -0.00634548,  0.00575911,
        0.00092733, -0.00162944,  0.00159545,  0.00386116,  0.01353705,
        0.00832087,  0.00846061])

In [56]:
for i, coef in enumerate(regressor.coef_):
  print(f'after standardization, feature {X.columns[i]}\' coefficient is {coef}')

after standardization, feature state' coefficient is 0.0072431341775669855
after standardization, feature county_code' coefficient is -0.007680431684311321
after standardization, feature Median Household Income' coefficient is 0.02848315934299903
after standardization, feature % Rural' coefficient is -0.006345477158138098
after standardization, feature Population_y' coefficient is 0.005759110687001514
after standardization, feature political_diff' coefficient is 0.0009273280179308594
after standardization, feature % less than 18 years of age' coefficient is -0.0016294371560461145
after standardization, feature % 65 and over' coefficient is 0.001595454532696914
after standardization, feature % Asian' coefficient is 0.003861157036376723
after standardization, feature % Black' coefficient is 0.013537048275175094
after standardization, feature % Hispanic' coefficient is 0.008320869441120466
after standardization, feature % Non-Hispanic White' coefficient is 0.00846061185443809


In [57]:
r_2 = 0
for i, column in enumerate(X.columns):
  regressor = LinearRegression().fit(X_scaled[:,:i+1], y) # get the first i columns of X
  r_2_new = regressor.score(X_scaled[:,:i+1], y) # the new R^2 we get by including one more covariate
  print(f'including column {column} increase R^2 by {r_2_new - r_2}')
  r_2 = r_2_new

including column state increase R^2 by 0.0003765472648872281
including column county_code increase R^2 by 0.0003394267906349224
including column Median Household Income increase R^2 by 0.5485604194610194
including column % Rural increase R^2 by 0.05091457402689403
including column Population_y increase R^2 by 0.02538062893652948
including column political_diff increase R^2 by 0.013220931563045824
including column % less than 18 years of age increase R^2 by 0.0010722136940091165
including column % 65 and over increase R^2 by 8.753840210484753e-07
including column % Asian increase R^2 by 0.0009433375705845126
including column % Black increase R^2 by 0.016161165040135428
including column % Hispanic increase R^2 by 0.0027014854705824654
including column % Non-Hispanic White increase R^2 by 0.0022486899935202365


# 3. Standardized Linear Regression on counties without shelter in place

In [58]:
data_no_SIP = data[data['SIP?'] == 0]

In [60]:
data_no_SIP

,state,county_code,diff_in_perc_at_home,SIP?,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
97,5.0,5001.0,0.002032,0,43887.0,34.7,17769,-0.305821,22.9,19.6,0.8,24.5,3.3,69.5
98,5.0,5003.0,0.004202,0,41127.0,51.7,20046,-0.365018,22.9,20.4,0.3,24.5,5.5,68.3
99,5.0,5005.0,-0.000820,0,39686.0,65.8,41619,-0.536072,17.7,30.9,0.5,0.4,2.5,94.5
100,5.0,5007.0,0.075433,0,68978.0,25.2,272608,-0.348571,26.3,13.4,4.1,1.8,16.9,73.0
101,5.0,5009.0,-0.000736,0,44482.0,62.2,37480,-0.584465,22.9,20.7,0.7,0.4,2.6,93.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,56.0,56035.0,0.042121,0,74570.0,100.0,9813,-0.643472,22.5,19.5,0.7,0.8,7.5,89.0
2991,56.0,56037.0,-0.018485,0,73315.0,10.9,43051,-0.535382,26.2,12.1,1.0,1.1,16.1,79.3
2993,56.0,56041.0,0.010157,0,63401.0,43.1,20299,-0.614926,28.8,14.1,0.5,0.7,9.2,87.4
2994,56.0,56043.0,-0.007825,0,55190.0,36.0,7885,-0.640377,22.7,21.7,0.8,0.5,14.1,82.4


In [61]:
# get covariates
X = data_no_SIP.drop(['diff_in_perc_at_home', 'SIP?'], axis=1)
# get response variable
y = data_no_SIP['diff_in_perc_at_home'] 

In [62]:
X

,state,county_code,Median Household Income,% Rural,Population_y,political_diff,% less than 18 years of age,% 65 and over,% Asian,% Black,% Hispanic,% Non-Hispanic White
97,5.0,5001.0,43887.0,34.7,17769,-0.305821,22.9,19.6,0.8,24.5,3.3,69.5
98,5.0,5003.0,41127.0,51.7,20046,-0.365018,22.9,20.4,0.3,24.5,5.5,68.3
99,5.0,5005.0,39686.0,65.8,41619,-0.536072,17.7,30.9,0.5,0.4,2.5,94.5
100,5.0,5007.0,68978.0,25.2,272608,-0.348571,26.3,13.4,4.1,1.8,16.9,73.0
101,5.0,5009.0,44482.0,62.2,37480,-0.584465,22.9,20.7,0.7,0.4,2.6,93.6
...,...,...,...,...,...,...,...,...,...,...,...,...
2990,56.0,56035.0,74570.0,100.0,9813,-0.643472,22.5,19.5,0.7,0.8,7.5,89.0
2991,56.0,56037.0,73315.0,10.9,43051,-0.535382,26.2,12.1,1.0,1.1,16.1,79.3
2993,56.0,56041.0,63401.0,43.1,20299,-0.614926,28.8,14.1,0.5,0.7,9.2,87.4
2994,56.0,56043.0,55190.0,36.0,7885,-0.640377,22.7,21.7,0.8,0.5,14.1,82.4


In [63]:
y

97      0.002032
98      0.004202
99     -0.000820
100     0.075433
101    -0.000736
          ...   
2990    0.042121
2991   -0.018485
2993    0.010157
2994   -0.007825
2995   -0.003361
Name: diff_in_perc_at_home, Length: 447, dtype: float64

In [64]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [65]:
regressor = LinearRegression().fit(X_scaled, y)
r_2 = regressor.score(X_scaled, y) # R^2
print(f'R^2 score is {r_2}')

R^2 score is 0.3067980861918215


In [66]:
regressor.coef_

array([-0.22790754,  0.21904827,  0.01626843,  0.00179075,  0.00554558,
        0.00269994, -0.00334213,  0.00129528, -0.00127749, -0.00961496,
       -0.0024937 , -0.01092017])

In [67]:
for i, coef in enumerate(regressor.coef_):
  print(f'after standardization, feature {X.columns[i]}\' coefficient is {coef}')

after standardization, feature state' coefficient is -0.22790753855864523
after standardization, feature county_code' coefficient is 0.21904826801031266
after standardization, feature Median Household Income' coefficient is 0.016268434062671464
after standardization, feature % Rural' coefficient is 0.001790748676264978
after standardization, feature Population_y' coefficient is 0.005545579721670222
after standardization, feature political_diff' coefficient is 0.0026999411011930305
after standardization, feature % less than 18 years of age' coefficient is -0.0033421343710521856
after standardization, feature % 65 and over' coefficient is 0.001295277714855242
after standardization, feature % Asian' coefficient is -0.00127748939824294
after standardization, feature % Black' coefficient is -0.009614956981858644
after standardization, feature % Hispanic' coefficient is -0.002493696695728749
after standardization, feature % Non-Hispanic White' coefficient is -0.010920170729077222


In [68]:
r_2 = 0
for i, column in enumerate(X.columns):
  regressor = LinearRegression().fit(X_scaled[:,:i+1], y) # get the first i columns of X
  r_2_new = regressor.score(X_scaled[:,:i+1], y) # the new R^2 we get by including one more covariate
  print(f'including column {column} increase R^2 by {r_2_new - r_2}')
  r_2 = r_2_new

including column state increase R^2 by 0.005057603351503959
including column county_code increase R^2 by 0.010152441600171593
including column Median Household Income increase R^2 by 0.19868826631888625
including column % Rural increase R^2 by 0.0006266413502102886
including column Population_y increase R^2 by 0.033922934193415855
including column political_diff increase R^2 by 0.011008881908698642
including column % less than 18 years of age increase R^2 by 8.131818770462651e-05
including column % 65 and over increase R^2 by 6.640328990070721e-05
including column % Asian increase R^2 by 7.736061213647893e-05
including column % Black increase R^2 by 0.01626381486570405
including column % Hispanic increase R^2 by 0.0003680330288340361
including column % Non-Hispanic White increase R^2 by 0.030484387484654984


# Conclusion

It seems like Median household income is the most important variable, at least from standardized linear regression models. This finding is consitent in both counties with shelter in place and place without shelter in place. 

Although it seems like in counties with shelter in place median household income has a bigger impact on people's change of behavior.